In [75]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np 
from math import sqrt 
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt

GET DATA

In [76]:
df = pd.read_csv('merge.csv').drop('Unnamed: 0', axis=1)

In [77]:
df.drop_duplicates(inplace=True, subset='id_food')

In [78]:
print(len(df))

7575


In [79]:
print(df.keys())

Index(['id_food', 'product_name', 'brands', 'nutriscore_grade',
       'main_category_en', 'energy_kcal_100g', 'fat_100g',
       'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'url', 'supermarket', 'price',
       'reference_price', 'reference_unit', 'medida_similitud'],
      dtype='object')


In [80]:
user_rating = df[['id_food','nutriscore_grade']]

In [81]:
print(len(user_rating))

7575


In [82]:
sample_df_usuario1 = df.sample(2525, random_state = 6)

In [83]:
sample_df_usuario1

,id_food,product_name,brands,nutriscore_grade,main_category_en,energy_kcal_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,url,supermarket,price,reference_price,reference_unit,medida_similitud
20275,27989,tortilla de patata,palacios,c,es:procesado,186.0,12.0,2.0,14.0,1.0,5.0,1.0,https://tienda.mercadona.es/product/80771/tort...,mercadona-es,1.75,2.92,kg,0.928571
19251,51362,yogur sabor coco,alteza,b,Cow milk yogurts,74.0,2.0,1.0,11.0,11.0,4.0,0.0,https://tienda.mercadona.es/product/20032/yogu...,mercadona-es,0.47,0.94,kg,0.923077
23858,51485,bebida de soja sabor vainilla,alteza,a,Vanilla soy milks,NaN,2.0,0.0,9.0,9.0,3.0,0.0,https://tienda.mercadona.es/product/29319/bebi...,mercadona-es,1.01,1.01,l,0.948718
17976,47224,refresco sin gas manzana,eroski,d,Apple soft drinks,26.0,0.0,0.0,6.0,6.0,0.0,0.0,https://tienda.mercadona.es/product/13606/refr...,mercadona-es,0.80,0.53,l,0.919444
9702,21070,salchichas frankfurt elpozo,elpozo,d,Sausages,233.0,19.0,7.0,5.0,1.0,12.0,2.0,https://www.carrefour.es/supermercado/salchich...,carrefour-es,2.10,3.09,kg,0.901887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20620,35771,preparado de marisco para paella,carrefour,b,es:marisco,80.0,1.0,1.0,1.0,1.0,17.0,1.0,https://www.carrefour.es/supermercado/preparad...,carrefour-es,2.19,5.47,kg,0.930612
23166,51735,crema de setas del bosque con champiñones,knorr,c,Canned plant-based foods,198.0,3.0,2.0,4.0,1.0,1.0,1.0,https://tienda.mercadona.es/product/8611/crema...,mercadona-es,1.89,3.78,l,0.945200
13315,38377,queso de oveja tierno,veritas,d,Sheep's-milk cheeses,377.0,32.0,23.0,4.0,1.0,27.0,1.0,https://www.carrefour.es/supermercado/queso-de...,carrefour-es,4.18,13.95,kg,0.907692
21007,33744,aceite de oliva virgen extra la flor de malaga,la flor de malaga,c,Extra-virgin olive oils,900.0,100.0,12.0,0.0,0.0,0.0,0.0,https://www.carrefour.es/supermercado/aceite-d...,carrefour-es,17.65,3.53,l,0.932035


Condicion usuario 1 basado en nutriscore "a" y "b"

In [84]:
def f(row):
    if row['nutriscore_grade'] == 'a':
        val = 4
    elif row['nutriscore_grade'] == 'b':
        val = 3
    else:
        val = 2
    return val


In [85]:
sample_df_usuario1['rating'] = sample_df_usuario1.apply(f, axis=1)

In [86]:
sample_df_usuario1.insert(0, 'Usuario', 1)

In [87]:
sample_df_usuario1

,Usuario,id_food,product_name,brands,nutriscore_grade,main_category_en,energy_kcal_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,url,supermarket,price,reference_price,reference_unit,medida_similitud,rating
20275,1,27989,tortilla de patata,palacios,c,es:procesado,186.0,12.0,2.0,14.0,1.0,5.0,1.0,https://tienda.mercadona.es/product/80771/tort...,mercadona-es,1.75,2.92,kg,0.928571,2
19251,1,51362,yogur sabor coco,alteza,b,Cow milk yogurts,74.0,2.0,1.0,11.0,11.0,4.0,0.0,https://tienda.mercadona.es/product/20032/yogu...,mercadona-es,0.47,0.94,kg,0.923077,3
23858,1,51485,bebida de soja sabor vainilla,alteza,a,Vanilla soy milks,NaN,2.0,0.0,9.0,9.0,3.0,0.0,https://tienda.mercadona.es/product/29319/bebi...,mercadona-es,1.01,1.01,l,0.948718,4
17976,1,47224,refresco sin gas manzana,eroski,d,Apple soft drinks,26.0,0.0,0.0,6.0,6.0,0.0,0.0,https://tienda.mercadona.es/product/13606/refr...,mercadona-es,0.80,0.53,l,0.919444,2
9702,1,21070,salchichas frankfurt elpozo,elpozo,d,Sausages,233.0,19.0,7.0,5.0,1.0,12.0,2.0,https://www.carrefour.es/supermercado/salchich...,carrefour-es,2.10,3.09,kg,0.901887,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20620,1,35771,preparado de marisco para paella,carrefour,b,es:marisco,80.0,1.0,1.0,1.0,1.0,17.0,1.0,https://www.carrefour.es/supermercado/preparad...,carrefour-es,2.19,5.47,kg,0.930612,3
23166,1,51735,crema de setas del bosque con champiñones,knorr,c,Canned plant-based foods,198.0,3.0,2.0,4.0,1.0,1.0,1.0,https://tienda.mercadona.es/product/8611/crema...,mercadona-es,1.89,3.78,l,0.945200,2
13315,1,38377,queso de oveja tierno,veritas,d,Sheep's-milk cheeses,377.0,32.0,23.0,4.0,1.0,27.0,1.0,https://www.carrefour.es/supermercado/queso-de...,carrefour-es,4.18,13.95,kg,0.907692,2
21007,1,33744,aceite de oliva virgen extra la flor de malaga,la flor de malaga,c,Extra-virgin olive oils,900.0,100.0,12.0,0.0,0.0,0.0,0.0,https://www.carrefour.es/supermercado/aceite-d...,carrefour-es,17.65,3.53,l,0.932035,2


Condicion usuario 2  nutriscore "b","c","d" 

In [88]:
sample_df_usuario2 = df.sample(2525, random_state = 4656)

In [89]:
def f(row):
    if row['nutriscore_grade'] == 'b':
        val = 4
    elif row['nutriscore_grade'] == 'c':
        val = 3
    elif row['nutriscore_grade'] == 'd':
        val = 3
    else:
        val = 1
    return val

In [90]:
sample_df_usuario2['rating'] = sample_df_usuario2.apply(f, axis=1)

In [91]:
sample_df_usuario2.insert(0, 'Usuario', 2)

Condicion usuario 3  nutriscore "a","b","c" 

In [92]:
sample_df_usuario3 = df.sample(2525, random_state = 5659)

In [93]:
def f(row):
    if row['nutriscore_grade'] == 'a':
        val = 4
    elif row['nutriscore_grade'] == 'b':
        val = 4
    elif row['nutriscore_grade'] == 'c':
        val = 3
    else:
        val = 2
    return val

In [94]:
sample_df_usuario3['rating'] = sample_df_usuario3.apply(f, axis=1)

In [95]:
sample_df_usuario3.insert(0, 'Usuario', 3)

Condicion usuario 4 

In [96]:
sample_df_usuario4 = df.sample(2525, random_state = 9897)

In [97]:
def f(row):
    if row['nutriscore_grade'] == 'a':
        val = 1
    elif row['nutriscore_grade'] == 'b':
        val = 1
    elif row['nutriscore_grade'] == 'c':
        val = 4
    else:
        val = 3
    return val

In [98]:
sample_df_usuario4['rating'] = sample_df_usuario4.apply(f, axis=1)

In [99]:
sample_df_usuario4.insert(0, 'Usuario', 4)

In [100]:
sample_df_usuario4

,Usuario,id_food,product_name,brands,nutriscore_grade,main_category_en,energy_kcal_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,url,supermarket,price,reference_price,reference_unit,medida_similitud,rating
4884,4,37927,queso gouda ecologico,asana,d,es:queso-gouda,342.0,27.0,19.0,0.0,0.0,24.0,1.0,https://www.carrefour.es/supermercado/queso-go...,carrefour-es,3.60,18.00,kg,0.891304,3
14791,4,37069,judias verdes planas,el corte ingles,a,Canned green beans,14.0,1.0,0.0,2.0,0.0,1.0,1.0,https://www.carrefour.es/supermercado/judias-v...,carrefour-es,0.99,0.99,kg,0.911111,1
20108,4,33472,huevos medianos frescos,spar,a,Eggs,141.0,10.0,3.0,1.0,1.0,13.0,0.0,https://tienda.mercadona.es/product/31505/huev...,mercadona-es,1.29,1.29,docena,0.927366,1
8321,4,38466,helado de chocolate,veritas,c,Chocolate-ice-creams,163.0,6.0,4.0,24.0,22.0,4.0,0.0,https://www.carrefour.es/supermercado/helado-d...,carrefour-es,3.99,5.32,kg,0.897436,4
6762,4,2051,jamon de cebo iberico 50% raza iberica,realvalle,e,Raw-cured-ham,327.0,23.0,9.0,0.0,0.0,30.0,5.0,https://www.carrefour.es/supermercado/jamon-de...,carrefour-es,95.00,12.67,kg,0.895000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22828,4,20115,atun en aceite de girasol,puerto gallego,c,Tunas in sunflower oil,271.0,20.0,4.0,1.0,0.0,21.0,1.0,https://tienda.mercadona.es/product/18086/atun...,mercadona-es,5.30,8.15,kg,0.942857,4
5367,4,34510,magdalenas 0% azucares añadidos,alimerka,c,Madeleines,419.0,25.0,3.0,53.0,2.0,5.0,1.0,https://www.carrefour.es/supermercado/magdalen...,carrefour-es,1.99,8.29,kg,0.892368,4
19701,4,26966,mozzarella fresca,consum,c,Mozzarella,203.0,15.0,15.0,1.0,1.0,16.0,1.0,https://tienda.mercadona.es/product/51050/mozz...,mercadona-es,0.80,6.40,kg,0.925926,4
23359,4,33241,sardinas en aceite de oliva,el barco pirata,c,Canned sardines,199.0,12.0,3.0,3.0,0.0,22.0,1.0,https://tienda.mercadona.es/product/18225/sard...,mercadona-es,1.30,7.93,kg,0.945946,4


Condicion usuario 5

In [101]:
sample_df_usuario5 = df.sample(2525, random_state = 68874)

In [102]:
def f(row):
    if row['nutriscore_grade'] == 'a':
        val = 4
    elif row['nutriscore_grade'] == 'b':
        val = 4
    elif row['nutriscore_grade'] == 'c':
        val = 4
    else:
        val = 3
    return val

In [103]:
sample_df_usuario5['rating'] = sample_df_usuario5.apply(f, axis=1)

In [104]:
sample_df_usuario5.insert(0, 'Usuario', 5)

Append

In [105]:
sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario2, ignore_index = True)

C:\Users\teteo\AppData\Local\Temp\ipykernel_12360\4151432817.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario2, ignore_index = True)


In [107]:
sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario3, ignore_index = True)

C:\Users\teteo\AppData\Local\Temp\ipykernel_12360\2208395475.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario3, ignore_index = True)


In [108]:
sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario4, ignore_index = True)

C:\Users\teteo\AppData\Local\Temp\ipykernel_12360\561810355.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario4, ignore_index = True)


In [109]:
sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario5, ignore_index = True)

C:\Users\teteo\AppData\Local\Temp\ipykernel_12360\3959134647.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_df_usuario1 = sample_df_usuario1.append(sample_df_usuario5, ignore_index = True)


In [110]:
sample_df_usuario1

,Usuario,id_food,product_name,brands,nutriscore_grade,main_category_en,energy_kcal_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,url,supermarket,price,reference_price,reference_unit,medida_similitud,rating
0,1,27989,tortilla de patata,palacios,c,es:procesado,186.0,12.0,2.0,14.0,1.0,5.0,1.0,https://tienda.mercadona.es/product/80771/tort...,mercadona-es,1.75,2.92,kg,0.928571,2
1,1,51362,yogur sabor coco,alteza,b,Cow milk yogurts,74.0,2.0,1.0,11.0,11.0,4.0,0.0,https://tienda.mercadona.es/product/20032/yogu...,mercadona-es,0.47,0.94,kg,0.923077,3
2,1,51485,bebida de soja sabor vainilla,alteza,a,Vanilla soy milks,NaN,2.0,0.0,9.0,9.0,3.0,0.0,https://tienda.mercadona.es/product/29319/bebi...,mercadona-es,1.01,1.01,l,0.948718,4
3,1,47224,refresco sin gas manzana,eroski,d,Apple soft drinks,26.0,0.0,0.0,6.0,6.0,0.0,0.0,https://tienda.mercadona.es/product/13606/refr...,mercadona-es,0.80,0.53,l,0.919444,2
4,1,21070,salchichas frankfurt elpozo,elpozo,d,Sausages,233.0,19.0,7.0,5.0,1.0,12.0,2.0,https://www.carrefour.es/supermercado/salchich...,carrefour-es,2.10,3.09,kg,0.901887,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12620,5,20521,chocolate a la taza,lacasa,e,Chocolates,500.0,26.0,15.0,58.0,50.0,7.0,0.0,https://tienda.mercadona.es/product/10005/choc...,mercadona-es,1.45,1.45,l,0.881650,3
12621,5,17817,leche desnatada president,president,b,Cow milks,43.0,0.0,0.0,5.0,5.0,3.0,0.0,https://www.carrefour.es/supermercado/leche-de...,carrefour-es,0.63,0.63,l,0.942857,4
12622,5,17420,tomate frito ecologico,apis,b,Tomatoes and their products,76.0,3.0,0.0,10.0,7.0,2.0,2.0,https://www.carrefour.es/supermercado/tomate-f...,carrefour-es,0.99,3.30,kg,0.902326,4
12623,5,36290,queso gruyere france,carrefour,d,French Gruyeres,396.0,32.0,21.0,1.0,1.0,27.0,1.0,https://www.carrefour.es/supermercado/queso-gr...,carrefour-es,4.75,23.75,kg,0.881633,3


In [111]:
user_rating = sample_df_usuario1[['Usuario','rating','id_food','nutriscore_grade']]

In [112]:
user_rating 

,Usuario,rating,id_food,nutriscore_grade
0,1,2,27989,c
1,1,3,51362,b
2,1,4,51485,a
3,1,2,47224,d
4,1,2,21070,d
...,...,...,...,...
12620,5,3,20521,e
12621,5,4,17817,b
12622,5,4,17420,b
12623,5,3,36290,d


In [113]:
user_rating.to_csv('Usuarios_reglas.csv')